In [1]:
""" LOAD DATA """

from scipy import misc
import glob
import os
import numpy as np

color_imgs = []
depth_imgs = []
for image_path in glob.glob('../data/resized/color/*.jpg'):
    path, file_name = os.path.split(image_path)
    c_im = misc.imread(image_path)
    d_im = misc.imread(os.path.join('../data/resized/depthCorrected', file_name[:-3]+'png'))
    color_imgs.append(c_im)
    depth_imgs.append(d_im)

/home/acar/.virtualenvs/gpuenv/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if sys.path[0] == '':
/home/acar/.virtualenvs/gpuenv/lib/python2.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]


In [2]:
""" load dummy data """
dummy_imgs = []
for image_path in glob.glob('../data/my_images/*.jpg'):
    path, file_name = os.path.split(image_path)
    c_im = misc.imread(image_path)
    c_im = misc.imresize(c_im, [256,256], 'bilinear')
    dummy_imgs.append(c_im)

/home/acar/.virtualenvs/gpuenv/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """
/home/acar/.virtualenvs/gpuenv/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [8]:
""" TRAIN TEST SPLIT """

from sklearn.model_selection import train_test_split

SUB_SAMPLE = 0.1
step = np.int32(1/SUB_SAMPLE)
x_train, x_test, y_train, y_test = train_test_split(np.asarray(color_imgs[0:-1:step]), np.asarray(depth_imgs[0:-1:step]), test_size=0.20, random_state=42)

In [9]:
""" NORMALIZATION """

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

y_train = y_train.astype('float32') / (2**16)
y_test = y_test.astype('float32') / (2**16)

y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], y_train.shape[2], 1))
y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], y_test.shape[2], 1))

x_dummy = np.asarray(dummy_imgs, dtype='float32') / 255.

In [12]:
""" DEFINE MODEL """

import numpy as np
import matplotlib.pyplot as plt
import keras

from keras import layers
from keras.layers import Input
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Concatenate
from keras.layers import Conv2D
from keras.models import Model
from keras.layers import Dense
from capsule_layers import PrimaryCap
from capsule_layers import CapsuleLayer
from capsule_layers import Length
from capsule_layers import Mask
from keras.layers import Lambda
from keras import layers, models, optimizers
from keras import callbacks
from keras.activations import softmax

in_shape = color_imgs[0].shape
input_img = Input(shape=in_shape)

ks = 3
nr = 21

x = layers.Conv2D(filters=512, kernel_size=ks, strides=1, padding='same', 
                      activation='relu', name='conv1')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
# x = layers.Conv2D(32, (ks, ks), activation='relu', padding='same')(x)
# x = layers.MaxPooling2D((2, 2), padding='same')(x)
primarycaps = PrimaryCap(x, dim_capsule=16, n_channels=1, kernel_size=ks, strides=2, padding='valid')
caps_layer_1 = CapsuleLayer(num_capsule=32, dim_capsule=16, num_routing=nr,
                         name='caps_layer_1')(primarycaps)
caps_layer_2 = CapsuleLayer(num_capsule=8, dim_capsule=64, num_routing=nr,
                         name='caps_layer_2')(caps_layer_1)
caps_layer_3 = CapsuleLayer(num_capsule=4, dim_capsule=128, num_routing=nr,
                         name='caps_layer_3')(caps_layer_2)
# caps_layer_4 = CapsuleLayer(num_capsule=8, dim_capsule=128, num_routing=nr,
#                          name='caps_layer_4')(caps_layer_3)

encoded = caps_layer_2
encoded = Flatten()(encoded)
# encoded = Mask()(encoded)


# x = layers.Conv2D(32, (ks, ks), activation='relu', padding='same')(input_img)
# x = layers.MaxPooling2D((2, 2), padding='same')(x)
# x = layers.Conv2D(16, (ks, ks), activation='relu', padding='same')(x)
# x = layers.MaxPooling2D((2, 2), padding='same')(x)
# x = layers.Conv2D(16, (ks, ks), activation='relu', padding='same')(x)
# # x = layers.MaxPooling2D((2, 2), padding='same')(x)
# # x = layers.Conv2D(320, (ks, ks), activation='relu', padding='same')(x)
# encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = encoded
# x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = layers.Reshape(target_shape=(8, 8, 32))(x)
x = layers.Conv2D(64, (ks, ks), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (ks, ks), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (ks, ks), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (ks, ks), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (ks, ks), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (ks, ks), activation='sigmoid', padding='same')(x)

# x = Dense(11552, activation='relu')(encoded)
# x = layers.Reshape(target_shape=(38, 38, 8))(x)
# x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
# x = layers.UpSampling2D((2, 2))(x)
# x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
# x = layers.UpSampling2D((2, 2))(x)
# x = layers.Conv2D(16, (3, 3), activation='relu')(x)
# x = layers.UpSampling2D((2, 2))(x)
# decoded = layers.Conv2D(NUM_CLASSES, (3, 3), activation='sigmoid', padding='same')(x)

base_lr = 3e-3
optim = keras.optimizers.Adam(lr=base_lr)

autoencoder = Model(input_img, decoded)
evalmodel = Model(input_img, [decoded, encoded])
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 256, 512)     14336     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 128, 128, 512)     0         
_________________________________________________________________
primarycap_conv2d (Conv2D)   (None, 63, 63, 16)        73744     
_________________________________________________________________
primarycap_reshape (Reshape) (None, 3969, 16)          0         
_________________________________________________________________
primarycap_squash (Lambda)   (None, 3969, 16)          0         
_________________________________________________________________
caps_layer_1 (CapsuleLayer)  (None, 32, 16)            32514048  
__________

In [13]:
cb = [callbacks.ModelCheckpoint('checkpoints/caps_ae_best_weights.hdf5',
                                             verbose=0,
                                             save_weights_only=True, save_best_only=True)]

history = autoencoder.fit(x_train, y_train, batch_size=8, epochs=200, 
                          validation_data=(x_test,y_test), shuffle=True,
                         callbacks=cb)

Train on 820 samples, validate on 205 samples
Epoch 1/200
820/820 [==============================] - 82s 100ms/step - loss: 0.4258 - val_loss: 0.3622
Epoch 2/200
820/820 [==============================] - 77s 94ms/step - loss: 0.3620 - val_loss: 0.3574
Epoch 3/200
820/820 [==============================] - 78s 95ms/step - loss: 0.3596 - val_loss: 0.3608
Epoch 4/200
820/820 [==============================] - 77s 94ms/step - loss: 0.3588 - val_loss: 0.3614
Epoch 5/200
820/820 [==============================] - 77s 94ms/step - loss: 0.3578 - val_loss: 0.3610
Epoch 6/200
820/820 [==============================] - 77s 94ms/step - loss: 0.3585 - val_loss: 0.3590
Epoch 7/200
820/820 [==============================] - 77s 94ms/step - loss: 0.3576 - val_loss: 0.3571
Epoch 8/200
820/820 [==============================] - 77s 94ms/step - loss: 0.3568 - val_loss: 0.3565
Epoch 9/200
820/820 [==============================] - 77s 94ms/step - loss: 0.3565 - val_loss: 0.3563
Epoch 10/200
820/820 [====

KeyboardInterrupt: 

In [ ]:
""" EVALUATE CAPSULE MODEL """

autoencoder.load_weights('checkpoints/caps_ae_best_weights.hdf5', by_name=True)
eval_model = Model(autoencoder.input, autoencoder.output)
eval_model.compile(optimizer='adam', loss='mse')
# eval_model.summary()
eval_model.load_weights('checkpoints/caps_ae_best_weights.hdf5', by_name=True)
eval_model.evaluate(x_test, y_test*327.675, batch_size=1, verbose=1)

In [ ]:
from keras.models import load_model

autoencoder.load_weights('checkpoints/caps_ae_best_weights.hdf5', by_name=True)
predicted = autoencoder.predict(x_test[0:20], verbose=1, batch_size=1)

In [ ]:
""" generate dummy output """

autoencoder.load_weights('checkpoints/caps_ae_best_weights.hdf5', by_name=True)
predicted = autoencoder.predict(x_dummy, verbose=1, batch_size=1)

In [ ]:
import numpy as np

def histeq(image, number_bins=256):
    # from http://www.janeriksolem.net/2009/06/histogram-equalization-with-python-and.html

    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), number_bins, normed=True)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = 255 * cdf / cdf[-1] # normalize

    # use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)

    return image_equalized.reshape(image.shape)

In [ ]:
from PIL import Image
n = np.minimum(len(predicted), 10)
o = 40
c = 1
plt.figure(figsize=(20, 60))
for i in range(n):    
    # display original img
    ax = plt.subplot(n, 3, i*3+1)
    plt.imshow(x_dummy[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.title("input")
    
    # display gt
    ax = plt.subplot(n, 3, i*3+2)
    plt.imshow(x_dummy[i,:,:,0])
#     plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.title("gt")
    
    # display reconstruction
    ax = plt.subplot(n, 3, i*3+3)
    plt.imshow(histeq(predicted[i,:,:,0]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.title("reconstructed")
plt.show()